In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy
import pickle
import xgboost as xgb
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.feature_selection import SelectPercentile, f_regression
from time import time, strftime, localtime

In [3]:
### Load data
train_df = pd.read_csv("../dataset/train.csv")
test_df = pd.read_csv("../dataset/test.csv")
submit_test_df = pd.read_csv("../dataset/submit_test.csv")

In [162]:
### Split data & process missing value

# Fill nan with "0.0"
train_fillna_df = train_df.iloc[:,1:-1].fillna(value=0.0)
test_fillna_df = test_df.iloc[:,1:].fillna(value=0.0)
all_fillna_df = pd.concat((train_fillna_df, test_fillna_df), axis = 0)

# Fill nan with "median"
#train_fillna_df = train_df.iloc[:,1:-1].fillna(train_df.iloc[:,1:-1].median())
#test_fillna_df = test_df.iloc[:,1:].fillna(test_df.iloc[:,1:].median())
#all_fillna_df = pd.concat((train_fillna_df, test_fillna_df), axis = 0)

# Seperate features fillna
#train_noIDnoLabel_df = train_df.iloc[:,1:-1]
#test_noID_df = test_df.iloc[:,1:]
#all_df = pd.concat((train_noIDnoLabel_df, test_noID_df), axis = 0)

label_name = "total_price"
label_df = pd.DataFrame(train_df[label_name])

In [140]:
#temp_df = all_fillna_df.copy()
temp_df = all_df.copy()

In [141]:
### Process miss value, feature: "village_income_median"
count = 0
VIM_mean = 0.0
t0 = time()

# Fill na with the mean of "city+town"
for index, row in all_df.iloc[:][(all_df["village_income_median"].isnull())].iterrows():
    count += 1
    VIM_mean = all_df["village_income_median"][(all_df["city"] == row["city"]) & (all_df["town"] == row["town"])].mean()
    # If the mean of "city+town" is null or zero, fill na with the mean of "city"
    if math.isnan(VIM_mean) or VIM_mean == 0:
        VIM_mean = all_df["village_income_median"][all_df["city"] == row["city"]].mean()
    temp_df["village_income_median"][(temp_df["village_income_median"].isnull()) & (temp_df["city"] == row["city"]) & (temp_df["town"] == row["town"])] = VIM_mean

print('spent time: %0.3fs' % (time() - t0), strftime("%Y-%m-%d %H:%M:%S", localtime()))
print ("count:", count)

C:\Users\me\Anaconda3\envs\tensorflow-gpu-py3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


spent time: 63.623s 2019-07-12 17:59:29
count: 1326


In [142]:
# Fill nan with "0.0"
all_fillna_df = temp_df.fillna(value=0.0)

In [362]:
### Drop no need features
all_process_df = all_fillna_df.drop(
    ["parking_area","parking_price"], axis=1)
print("all_process_df.shape:", all_process_df.shape)

all_process_df.shape: (70000, 231)


In [430]:
### Create features
# [house_age: the age of house on purchased date, ((成交日 - 完工日) / 3650) ((txn_dt - building_complete_dt) / 3650)]
all_createCol_df = all_fillna_df.assign(house_age = round((all_fillna_df["txn_dt"] - all_fillna_df["building_complete_dt"]) / 3650, 0))

# [txn_year: the year of trade, 成交日 / 365, txn_dt / 365]
all_createCol_df = all_createCol_df.assign(txn_year = round((all_fillna_df["txn_dt"] / 365), 0))

# [floor_of_building: floor of building, 交易樓層 / 總樓層(txn_floor / total_floor)]，如 txn_floor 為 0，代表建物不為一層，以總樓層帶入
all_createCol_df = all_createCol_df.assign(floor_of_building = all_fillna_df["txn_floor"] / all_fillna_df["total_floor"])
all_createCol_df["floor_of_building"][all_createCol_df["txn_floor"] == 0] = all_fillna_df["total_floor"]

inf = float("inf")

# [sales_per_unit_area 坪效: 一坪土地可蓋出多少坪可銷售面積, (建物面積 / 土地面積), (building_area / land_area)]
all_createCol_df = all_createCol_df.assign(sales_per_unit_area = round(all_createCol_df["building_area"] / all_createCol_df["land_area"], 1))
all_createCol_df["sales_per_unit_area"][all_createCol_df["sales_per_unit_area"] == inf] = all_createCol_df["sales_per_unit_area"].median()

# [power_pos_rate 強化正相關率: 將會提高房價的因素加總，再減會減低房價的因素, (所在縣市出生率 + 所在縣市結婚率 - 所在縣市死亡率), (born_rate + marriage_rate - death_rate)]
all_createCol_df = all_createCol_df.assign(power_pos_rate = all_createCol_df["born_rate"] + all_createCol_df["marriage_rate"] - all_createCol_df["death_rate"])
#all_createCol_df = all_fillna_df.assign(power_pos_rate = all_fillna_df["born_rate"] + all_fillna_df["marriage_rate"] - all_fillna_df["death_rate"])

# [high_edu_rate 各縣市高學歷率: 各縣市的高學歷加總比率(大學含以上)]
#all_createCol_df = all_createCol_df.assign(high_edu_rate = round(all_createCol_df["doc_rate"] + all_createCol_df["master_rate"] + all_createCol_df["bachelor_rate"], 3))

# [low_edu_rate 各縣市低學歷率: 各縣市的低學歷加總比率(專科+高中含以下)]
#all_createCol_df = all_createCol_df.assign(low_edu_rate = round(all_createCol_df["jobschool_rate"] + all_createCol_df["highschool_rate"] + all_createCol_df["junior_rate"] + all_createCol_df["elementary_rate"], 3))

# [is_apartment_first_top 是否為公寓的一樓跟頂樓: 公寓的一樓跟頂樓最貴]
all_createCol_df = all_createCol_df.assign(is_apartment_first_top = 0.0)
all_createCol_df["is_apartment_first_top"][(all_createCol_df["building_type"] == 1) & ((all_createCol_df["txn_floor"] == 1) |  (all_createCol_df["txn_floor"] == all_createCol_df["total_floor"]))] = 1

# [is_0_3_high_floor 是否為電梯大樓、華廈的高樓層(7樓以上): 樓層越高越貴]
#all_createCol_df = all_createCol_df.assign(is_0_3_high_floor = 0.0)
#all_createCol_df["is_0_3_high_floor"][((all_createCol_df["building_type"] == 0) | (all_createCol_df["building_type"] == 3)) & (all_createCol_df["txn_floor"] >= 7)] = 1

# [_0_3_high_floor_rate 電梯大樓、華廈的高樓層高度比率(7樓以上): 樓層越高越貴]
#all_createCol_df = all_createCol_df.assign(_0_3_high_floor_rate = 0.0)
#all_createCol_df["_0_3_high_floor_rate"][((all_createCol_df["building_type"] == 0) | (all_createCol_df["building_type"] == 3)) & (all_createCol_df["txn_floor"] >= 7)] = (all_fillna_df["txn_floor"] / all_fillna_df["total_floor"])

C:\Users\me\Anaconda3\envs\tensorflow-gpu-py3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [434]:
all_createCol_df[["is_apartment_first_top"]].head()

,is_apartment_first_top
0,0.0
1,1.0
2,1.0
3,0.0
4,0.0


In [435]:
all_createCol_df.shape

(70000, 239)

In [436]:
#all_process_df = all_df.copy()
#all_process_df = all_selection_df.copy()
#all_process_df = all_fillna_df.copy()
all_process_df = all_createCol_df.copy()

In [437]:
# [No-ohe version]
### min max scalar
# preserve column name after scaler
min_max_scaler = preprocessing.MinMaxScaler()
mms_features_df = pd.DataFrame(min_max_scaler.fit_transform(all_process_df))

## Split to train & test data
train_num = train_df.shape[0]
#train_num = train_remove_outlier_df.shape[0]
mms_train_process_df = mms_features_df.iloc[:train_num,:]
mms_test_process_df = mms_features_df.iloc[train_num:,:]

print("mms_train_process_df.shape:", mms_train_process_df.shape)
print("mms_test_process_df.shape:", mms_test_process_df.shape)

mms_train_process_df.shape: (60000, 239)
mms_test_process_df.shape: (10000, 239)


In [438]:
### Shuffle data

#shuff_train_all = shuffle(pd.concat((mms_train_process_df, label_df), axis=1))
# No min max scalar
#shuff_train_all = shuffle(pd.concat((train_process_df, label_df), axis=1))
# Drop unimportant features
#shuff_train_all = shuffle(pd.concat((temp_train_df, label_df), axis=1))
# min max
shuff_train_all = shuffle(pd.concat((mms_train_process_df, label_df), axis=1), random_state=7)
#shuff_train_all = shuffle(pd.concat((mms_train_process_df, label_df), axis=1))
# Drop & min max & remove outliers
#shuff_train_all = shuffle(pd.concat((mms_train_process_df, label_remove_outliers_df), axis=1), random_state=0)
# Drop & remove outliers
#shuff_train_all = shuffle(pd.concat((train_drop_df, label_remove_outliers_df), axis=1), random_state=0)


# process train and validation num
tv_num = round(shuff_train_all.shape[0] * 0.85)

### Split data
# training data
X_train = shuff_train_all.iloc[:tv_num,:-1]
y_train = shuff_train_all.iloc[:tv_num,-1:]

print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)

# validation data
X_val = shuff_train_all.iloc[tv_num:,:-1]
y_val = shuff_train_all.iloc[tv_num:,-1:]

print("X_val.shape:", X_val.shape)
print("y_val.shape:", y_val.shape)

X_train.shape: (51000, 239)
y_train.shape: (51000, 1)
X_val.shape: (9000, 239)
y_val.shape: (9000, 1)


In [439]:
### Transfter into data matrix
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalidation = xgb.DMatrix(X_val, label=y_val)
#dtest = xgb.DMatrix(mms_test_process_df)
# No min max scaler
#dtest = xgb.DMatrix(test_process_df)
# Drop unimportant features
#dtest = xgb.DMatrix(temp_test_df)
# Drop & min max
dtest = xgb.DMatrix(mms_test_process_df)
# Drop
#dtest = xgb.DMatrix(test_drop_df)

In [452]:
### Booster parameters

param = {'objective': 'reg:squarederror', 'colsample_bytree': 0.8, 'subsample': 0.5, 'max_depth': 10, 'eta': 0.01, 
        'min_child_weight': 0.3, 'reg_alpha': 0.01, 'reg_lambda': 1, 'gamma': 0.0001, 'verbosity': 1, 'random_state': 7, 
        'nthread': -1, 'tree_method': 'gpu_hist', 'booster': 'gbtree'}
param['eval_metric'] = ['rmse']

### Training model
print('開始時間：', strftime("%Y-%m-%d %H:%M:%S", localtime()))
t0 = time()

#evallist = [(dtrain, 'train'), (dvalidation, 'eval')]
evallist = [(dvalidation, 'eval'), (dtrain, 'train')]
num_round = 200000
#bst = xgb.train(param, dtrain, num_round, evallist)
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10)

print('spent time: %0.3fs' % (time() - t0), strftime("%Y-%m-%d %H:%M:%S", localtime()))

### Predict
print('開始時間：', strftime("%Y-%m-%d %H:%M:%S", localtime()))
t0 = time()

xgb_pred = bst.predict(dtest)
y_test = bst.predict(dtrain)

print('spent time: %0.3fs' % (time() - t0), strftime("%Y-%m-%d %H:%M:%S", localtime()))

# validation
RMSE = np.sqrt(mean_squared_error(y_train, y_test))
print("{:,}".format(RMSE.round(4)))

開始時間： 2019-07-14 23:26:42
[0]	eval-rmse:7.52728e+07	train-rmse:5.22946e+07
Multiple eval metrics have been passed: 'train-rmse' will be used for early stopping.

Will train until train-rmse hasn't improved in 10 rounds.
[1]	eval-rmse:7.49747e+07	train-rmse:5.19739e+07
[2]	eval-rmse:7.46437e+07	train-rmse:5.16091e+07
[3]	eval-rmse:7.41839e+07	train-rmse:5.12624e+07
[4]	eval-rmse:7.37825e+07	train-rmse:5.09032e+07
[5]	eval-rmse:7.35842e+07	train-rmse:5.06796e+07
[6]	eval-rmse:7.33686e+07	train-rmse:5.03488e+07
[7]	eval-rmse:7.30499e+07	train-rmse:5.00666e+07
[8]	eval-rmse:7.27936e+07	train-rmse:4.98348e+07
[9]	eval-rmse:7.25743e+07	train-rmse:4.95573e+07
[10]	eval-rmse:7.23682e+07	train-rmse:4.9294e+07
[11]	eval-rmse:7.20462e+07	train-rmse:4.90205e+07
[12]	eval-rmse:7.17522e+07	train-rmse:4.87228e+07
[13]	eval-rmse:7.14256e+07	train-rmse:4.84126e+07
[14]	eval-rmse:7.10975e+07	train-rmse:4.81284e+07
[15]	eval-rmse:7.08981e+07	train-rmse:4.78508e+07
[16]	eval-rmse:7.05628e+07	train-rmse:4.

[161]	eval-rmse:4.52585e+07	train-rmse:2.24248e+07
[162]	eval-rmse:4.51323e+07	train-rmse:2.23317e+07
[163]	eval-rmse:4.50421e+07	train-rmse:2.2263e+07
[164]	eval-rmse:4.49908e+07	train-rmse:2.21757e+07
[165]	eval-rmse:4.48049e+07	train-rmse:2.20699e+07
[166]	eval-rmse:4.4698e+07	train-rmse:2.19785e+07
[167]	eval-rmse:4.46633e+07	train-rmse:2.18891e+07
[168]	eval-rmse:4.45054e+07	train-rmse:2.17847e+07
[169]	eval-rmse:4.44367e+07	train-rmse:2.17037e+07
[170]	eval-rmse:4.44026e+07	train-rmse:2.16491e+07
[171]	eval-rmse:4.43087e+07	train-rmse:2.15314e+07
[172]	eval-rmse:4.4214e+07	train-rmse:2.14218e+07
[173]	eval-rmse:4.41243e+07	train-rmse:2.13175e+07
[174]	eval-rmse:4.40498e+07	train-rmse:2.12051e+07
[175]	eval-rmse:4.40046e+07	train-rmse:2.11267e+07
[176]	eval-rmse:4.3934e+07	train-rmse:2.10276e+07
[177]	eval-rmse:4.3884e+07	train-rmse:2.09725e+07
[178]	eval-rmse:4.3713e+07	train-rmse:2.08481e+07
[179]	eval-rmse:4.36415e+07	train-rmse:2.07777e+07
[180]	eval-rmse:4.35543e+07	train-rms

[323]	eval-rmse:3.555e+07	train-rmse:1.27029e+07
[324]	eval-rmse:3.54957e+07	train-rmse:1.26749e+07
[325]	eval-rmse:3.5467e+07	train-rmse:1.26221e+07
[326]	eval-rmse:3.54252e+07	train-rmse:1.25898e+07
[327]	eval-rmse:3.54303e+07	train-rmse:1.25711e+07
[328]	eval-rmse:3.53954e+07	train-rmse:1.25272e+07
[329]	eval-rmse:3.53852e+07	train-rmse:1.24943e+07
[330]	eval-rmse:3.53526e+07	train-rmse:1.24728e+07
[331]	eval-rmse:3.53062e+07	train-rmse:1.24137e+07
[332]	eval-rmse:3.52907e+07	train-rmse:1.23759e+07
[333]	eval-rmse:3.52397e+07	train-rmse:1.23209e+07
[334]	eval-rmse:3.51892e+07	train-rmse:1.22657e+07
[335]	eval-rmse:3.51729e+07	train-rmse:1.22148e+07
[336]	eval-rmse:3.51186e+07	train-rmse:1.21769e+07
[337]	eval-rmse:3.50813e+07	train-rmse:1.21514e+07
[338]	eval-rmse:3.50462e+07	train-rmse:1.21197e+07
[339]	eval-rmse:3.50144e+07	train-rmse:1.2078e+07
[340]	eval-rmse:3.49816e+07	train-rmse:1.20252e+07
[341]	eval-rmse:3.49813e+07	train-rmse:1.20085e+07
[342]	eval-rmse:3.49707e+07	train-r

[485]	eval-rmse:3.18764e+07	train-rmse:8.26848e+06
[486]	eval-rmse:3.18755e+07	train-rmse:8.2461e+06
[487]	eval-rmse:3.18537e+07	train-rmse:8.23531e+06
[488]	eval-rmse:3.18307e+07	train-rmse:8.21047e+06
[489]	eval-rmse:3.18307e+07	train-rmse:8.20055e+06
[490]	eval-rmse:3.18327e+07	train-rmse:8.19147e+06
[491]	eval-rmse:3.18027e+07	train-rmse:8.18015e+06
[492]	eval-rmse:3.17918e+07	train-rmse:8.16218e+06
[493]	eval-rmse:3.17661e+07	train-rmse:8.13301e+06
[494]	eval-rmse:3.17459e+07	train-rmse:8.11235e+06
[495]	eval-rmse:3.17237e+07	train-rmse:8.09984e+06
[496]	eval-rmse:3.17075e+07	train-rmse:8.07389e+06
[497]	eval-rmse:3.16844e+07	train-rmse:8.0621e+06
[498]	eval-rmse:3.16699e+07	train-rmse:8.03891e+06
[499]	eval-rmse:3.16424e+07	train-rmse:8.02056e+06
[500]	eval-rmse:3.16284e+07	train-rmse:8.01e+06
[501]	eval-rmse:3.16115e+07	train-rmse:7.99077e+06
[502]	eval-rmse:3.1594e+07	train-rmse:7.96697e+06
[503]	eval-rmse:3.15751e+07	train-rmse:7.94012e+06
[504]	eval-rmse:3.15578e+07	train-rms

[647]	eval-rmse:3.04356e+07	train-rmse:6.17869e+06
[648]	eval-rmse:3.04382e+07	train-rmse:6.1632e+06
[649]	eval-rmse:3.04298e+07	train-rmse:6.14760e+06
[650]	eval-rmse:3.04252e+07	train-rmse:6.14128e+06
[651]	eval-rmse:3.04077e+07	train-rmse:6.13152e+06
[652]	eval-rmse:3.04041e+07	train-rmse:6.11602e+06
[653]	eval-rmse:3.04063e+07	train-rmse:6.1042e+06
[654]	eval-rmse:3.04023e+07	train-rmse:6.09716e+06
[655]	eval-rmse:3.0405e+07	train-rmse:6.08879e+06
[656]	eval-rmse:3.03925e+07	train-rmse:6.08286e+06
[657]	eval-rmse:3.03823e+07	train-rmse:6.07969e+06
[658]	eval-rmse:3.03821e+07	train-rmse:6.07032e+06
[659]	eval-rmse:3.03758e+07	train-rmse:6.06484e+06
[660]	eval-rmse:3.03703e+07	train-rmse:6.04915e+06
[661]	eval-rmse:3.03719e+07	train-rmse:6.03672e+06
[662]	eval-rmse:3.03625e+07	train-rmse:6.02674e+06
[663]	eval-rmse:3.03497e+07	train-rmse:6.0198e+06
[664]	eval-rmse:3.03398e+07	train-rmse:6.01078e+06
[665]	eval-rmse:3.03327e+07	train-rmse:5.99539e+06
[666]	eval-rmse:3.03235e+07	train-r

...

[3356]	eval-rmse:2.93481e+07	train-rmse:1.48682e+06
[3357]	eval-rmse:2.93479e+07	train-rmse:1.48584e+06
[3358]	eval-rmse:2.9348e+07	train-rmse:1.48534e+06
[3359]	eval-rmse:2.93481e+07	train-rmse:1.48506e+06
[3360]	eval-rmse:2.93479e+07	train-rmse:1.48455e+06
[3361]	eval-rmse:2.93479e+07	train-rmse:1.48393e+06
[3362]	eval-rmse:2.93477e+07	train-rmse:1.48333e+06
[3363]	eval-rmse:2.93477e+07	train-rmse:1.48302e+06
[3364]	eval-rmse:2.93477e+07	train-rmse:1.48258e+06
[3365]	eval-rmse:2.93478e+07	train-rmse:1.48233e+06
[3366]	eval-rmse:2.93483e+07	train-rmse:1.48165e+06
[3367]	eval-rmse:2.93486e+07	train-rmse:1.48126e+06
[3368]	eval-rmse:2.9349e+07	train-rmse:1.48085e+06
[3369]	eval-rmse:2.93494e+07	train-rmse:1.48e+06
[3370]	eval-rmse:2.93496e+07	train-rmse:1.47973e+06
[3371]	eval-rmse:2.93494e+07	train-rmse:1.47876e+06
[3372]	eval-rmse:2.93494e+07	train-rmse:1.47855e+06
[3373]	eval-rmse:2.93494e+07	train-rmse:1.47805e+06
[3374]	eval-rmse:2.93496e+07	train-rmse:1.47784e+06
[3375]	eval-rmse:

[3515]	eval-rmse:2.93505e+07	train-rmse:1.40292e+06
[3516]	eval-rmse:2.93504e+07	train-rmse:1.40243e+06
[3517]	eval-rmse:2.93505e+07	train-rmse:1.40218e+06
[3518]	eval-rmse:2.93506e+07	train-rmse:1.40178e+06
[3519]	eval-rmse:2.93506e+07	train-rmse:1.40149e+06
[3520]	eval-rmse:2.93507e+07	train-rmse:1.40092e+06
[3521]	eval-rmse:2.93505e+07	train-rmse:1.40045e+06
[3522]	eval-rmse:2.93503e+07	train-rmse:1.39992e+06
[3523]	eval-rmse:2.935e+07	train-rmse:1.3987e+06
[3524]	eval-rmse:2.935e+07	train-rmse:1.39815e+06
[3525]	eval-rmse:2.93498e+07	train-rmse:1.39774e+06
[3526]	eval-rmse:2.93496e+07	train-rmse:1.39726e+06
[3527]	eval-rmse:2.93494e+07	train-rmse:1.39695e+06
[3528]	eval-rmse:2.93501e+07	train-rmse:1.39674e+06
[3529]	eval-rmse:2.93497e+07	train-rmse:1.39645e+06
[3530]	eval-rmse:2.93498e+07	train-rmse:1.39567e+06
[3531]	eval-rmse:2.93499e+07	train-rmse:1.39504e+06
[3532]	eval-rmse:2.93499e+07	train-rmse:1.39455e+06
[3533]	eval-rmse:2.93499e+07	train-rmse:1.39442e+06
[3534]	eval-rmse:

[3674]	eval-rmse:2.93454e+07	train-rmse:1.32507e+06
[3675]	eval-rmse:2.93453e+07	train-rmse:1.32478e+06
[3676]	eval-rmse:2.93451e+07	train-rmse:1.32429e+06
[3677]	eval-rmse:2.93455e+07	train-rmse:1.32406e+06
[3678]	eval-rmse:2.93454e+07	train-rmse:1.32345e+06
[3679]	eval-rmse:2.93453e+07	train-rmse:1.32329e+06
[3680]	eval-rmse:2.93459e+07	train-rmse:1.32305e+06
[3681]	eval-rmse:2.93459e+07	train-rmse:1.32265e+06
[3682]	eval-rmse:2.93458e+07	train-rmse:1.32239e+06
[3683]	eval-rmse:2.93458e+07	train-rmse:1.32184e+06
[3684]	eval-rmse:2.93458e+07	train-rmse:1.32161e+06
[3685]	eval-rmse:2.93458e+07	train-rmse:1.32124e+06
[3686]	eval-rmse:2.93456e+07	train-rmse:1.32075e+06
[3687]	eval-rmse:2.93453e+07	train-rmse:1.3205e+06
[3688]	eval-rmse:2.93454e+07	train-rmse:1.32001e+06
[3689]	eval-rmse:2.93454e+07	train-rmse:1.31967e+06
[3690]	eval-rmse:2.93455e+07	train-rmse:1.3192e+06
[3691]	eval-rmse:2.93455e+07	train-rmse:1.3187e+06
[3692]	eval-rmse:2.93455e+07	train-rmse:1.31789e+06
[3693]	eval-rms

[3833]	eval-rmse:2.9346e+07	train-rmse:1.25628e+06
[3834]	eval-rmse:2.9346e+07	train-rmse:1.25604e+06
[3835]	eval-rmse:2.9346e+07	train-rmse:1.25559e+06
[3836]	eval-rmse:2.9346e+07	train-rmse:1.25542e+06
[3837]	eval-rmse:2.9346e+07	train-rmse:1.2548e+06
[3838]	eval-rmse:2.9346e+07	train-rmse:1.25455e+06
[3839]	eval-rmse:2.9346e+07	train-rmse:1.25403e+06
[3840]	eval-rmse:2.93461e+07	train-rmse:1.25372e+06
[3841]	eval-rmse:2.9346e+07	train-rmse:1.25311e+06
[3842]	eval-rmse:2.9346e+07	train-rmse:1.25281e+06
[3843]	eval-rmse:2.93461e+07	train-rmse:1.2524e+06
[3844]	eval-rmse:2.9346e+07	train-rmse:1.25185e+06
[3845]	eval-rmse:2.9346e+07	train-rmse:1.25121e+06
[3846]	eval-rmse:2.93459e+07	train-rmse:1.25093e+06
[3847]	eval-rmse:2.9346e+07	train-rmse:1.25048e+06
[3848]	eval-rmse:2.9346e+07	train-rmse:1.25008e+06
[3849]	eval-rmse:2.93463e+07	train-rmse:1.24976e+06
[3850]	eval-rmse:2.93463e+07	train-rmse:1.24926e+06
[3851]	eval-rmse:2.93464e+07	train-rmse:1.24856e+06
[3852]	eval-rmse:2.93464e+0

[3992]	eval-rmse:2.93472e+07	train-rmse:1.19137e+06
[3993]	eval-rmse:2.93472e+07	train-rmse:1.19105e+06
[3994]	eval-rmse:2.93472e+07	train-rmse:1.19072e+06
[3995]	eval-rmse:2.93473e+07	train-rmse:1.19041e+06
[3996]	eval-rmse:2.93472e+07	train-rmse:1.19001e+06
[3997]	eval-rmse:2.93473e+07	train-rmse:1.18953e+06
[3998]	eval-rmse:2.93473e+07	train-rmse:1.18912e+06
[3999]	eval-rmse:2.93473e+07	train-rmse:1.18855e+06
spent time: 143.317s 2019-07-14 23:29:06
開始時間： 2019-07-14 23:29:06
spent time: 1.141s 2019-07-14 23:29:07
1,188,551.6514


In [450]:
### Validation
RMSE_train = np.sqrt(mean_squared_error(y_train, y_test))
print('RMSE_train:', "{:,}".format(RMSE_train.round(4)))

y_val_test = bst.predict(dvalidation)
RMSE_val = np.sqrt(mean_squared_error(y_val, y_val_test))
print('RMSE_val:', "{:,}".format(RMSE_val.round(4)))

RMSE_train: 1,484,015.7709
RMSE_val: 29,181,111.7766


In [453]:
### Describe data
print("y_train:", pd.DataFrame(y_train).describe())
print("y_test:", pd.DataFrame(y_test).describe())
print("xgb_pred:", pd.DataFrame(xgb_pred).describe(), "\n")
print("number of xgb_pred are negative:", len(xgb_pred[xgb_pred < 0]))

y_train:         total_price
count  5.100000e+04
mean   1.284103e+07
std    5.108065e+07
min    2.261495e+05
25%    2.436585e+06
50%    5.240482e+06
75%    1.123932e+07
max    4.357039e+09
y_test:                   0
count  5.100000e+04
mean   1.284045e+07
std    5.101806e+07
min   -5.353824e+06
25%    2.635457e+06
50%    5.391494e+06
75%    1.117924e+07
max    4.355997e+09
xgb_pred:                   0
count  1.000000e+04
mean   1.319855e+07
std    4.696391e+07
min   -2.750140e+07
25%    2.580645e+06
50%    5.396035e+06
75%    1.103688e+07
max    2.038183e+09 

number of xgb_pred are negative: 36


In [69]:
### Save model
with open('../model/xgb_tree_1.pkl', 'wb') as f:
    pickle.dump(bst, f)

In [455]:
### Submission

import os
import re

# 取得 submit 檔最大值的下一個數字
def getMaxFileNum():
    max = 0
    for f in os.listdir("../submit"):
        fileNum = re.findall('\d+', f)
        if len(fileNum):
            if int(fileNum[0]) > max:
                max = int(fileNum[0])
    return str(max + 1)

filePath = "../submit/submit_test_" + getMaxFileNum() + ".csv"

pred_df = pd.DataFrame(np.array(xgb_pred), columns=["total_price"])
ans_df = pd.merge(submit_test_df["building_id"].to_frame(), pred_df, left_index=True, right_index=True, how="outer")
ans_df.to_csv(filePath,sep=",",index=False,encoding="UTF-8")

print('filePath:', filePath)

filePath: ../submit/submit_test_72.csv
